In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import numpy as np
import random

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [2]:
(train_dataset,test_dataset),ds_info = tfds.load('imdb_reviews',
                                                 with_info = True,
                                         split = ['train','test'],
                                                  as_supervised = True,
                                         shuffle_files = True,
                                         batch_size = 128

                                                 )

In [3]:
def text_vectorization(dataset,vectorizer):
  """
  Uses TextVectorization Layer outside of the model.
  The purpose is to make the NN model suitable to .h5 format.

  Parameters
  ----------
  dataset: Tensorflow dataset containing data and labels.
  vectorizer: A TextVectorization layer adapted to train_dataset
  """
  #conduct all the process in CPU to CPU-GPU conflict
  with tf.device("/cpu:0"):
    outputs = []
    label_list = []

    #tokenize and pad the data and store it
    for x, y in dataset:
        output = vectorizer(x)
        outputs.append(output)
        label_list.append(y)

    #concatenate the labels and data
    X_vectorized = tf.concat(outputs, axis=0)
    Y_labels = tf.concat(label_list, axis=0)

    vectorized = tf.data.Dataset.from_tensor_slices((X_vectorized, Y_labels)).batch(32).prefetch(tf.data.AUTOTUNE)
    return vectorized

def define_callbacks(model):
  es = tf.keras.callbacks.EarlyStopping(patience = 5,verbose = 1, restore_best_weights = True)
  mc = tf.keras.callbacks.ModelCheckpoint(filepath = f"./ModelCheckpoints/{model.name}.ckpt",
                                         save_best_only = True,
                                         save_weights_only = True)
  tb = tf.keras.callbacks.TensorBoard(log_dir = f"./TensorboardLogs/{model.name}")

  return es,mc,tb

In [4]:
#get the vocab length
vectorizer = layers.TextVectorization(max_tokens = None)
vectorizer.adapt(train_dataset.map(lambda x,y: x))
total_words = vectorizer.vocabulary_size()

In [5]:
#save the length of the sequences to find the output sequence length
text_lengths = []
train_dataset_unbatched = train_dataset.unbatch()
for text,label in train_dataset_unbatched: #unbatch the dataset
  text_lengths.append(len(text.numpy().split())) #append the sentence length

In [6]:
#create model and history dictionaries
models = {}
histories = {}

In [7]:
MAX_TOKENS = 20000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 128
MODEL_NAME = "model1"
HISTORY_NAME = 'history1'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)

vectorizer.adapt(train_dataset.map(lambda x,y: x))
#preprocess the data
train_dataset_vectorized1 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized1 = text_vectorization(test_dataset,vectorizer)


inputs = tf.keras.layers.Input(shape = (598,))

x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)


#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized1,validation_data = test_dataset_vectorized1,epochs = 5,
                    )



Epoch 1/5
782/782 [==============================] - 109s 137ms/step - loss: 0.6057 - accuracy: 0.7140 - val_loss: 0.4894 - val_accuracy: 0.8165
Epoch 2/5
782/782 [==============================] - 5s 6ms/step - loss: 0.3901 - accuracy: 0.8631 - val_loss: 0.3611 - val_accuracy: 0.8650
Epoch 3/5
782/782 [==============================] - 5s 7ms/step - loss: 0.2939 - accuracy: 0.8950 - val_loss: 0.3164 - val_accuracy: 0.8802
Epoch 4/5
782/782 [==============================] - 6s 8ms/step - loss: 0.2440 - accuracy: 0.9125 - val_loss: 0.2961 - val_accuracy: 0.8854
Epoch 5/5
782/782 [==============================] - 5s 7ms/step - loss: 0.2096 - accuracy: 0.9267 - val_loss: 0.2866 - val_accuracy: 0.8872


Add another Dense Layer -> Not worked. Increased overfitting.

In [8]:
MAX_TOKENS = 20000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 128
MODEL_NAME = "model2"
HISTORY_NAME = 'history2'


inputs = tf.keras.layers.Input(shape = (598,))

x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized1,validation_data = test_dataset_vectorized1,epochs = 5,
                   )


Epoch 1/5
782/782 [==============================] - 78s 99ms/step - loss: 0.4440 - accuracy: 0.7880 - val_loss: 0.3000 - val_accuracy: 0.8807
Epoch 2/5
782/782 [==============================] - 6s 7ms/step - loss: 0.2212 - accuracy: 0.9142 - val_loss: 0.2848 - val_accuracy: 0.8849
Epoch 3/5
782/782 [==============================] - 6s 8ms/step - loss: 0.1579 - accuracy: 0.9423 - val_loss: 0.2976 - val_accuracy: 0.8850
Epoch 4/5
782/782 [==============================] - 5s 7ms/step - loss: 0.1180 - accuracy: 0.9589 - val_loss: 0.3413 - val_accuracy: 0.8786
Epoch 5/5
782/782 [==============================] - 7s 9ms/step - loss: 0.0927 - accuracy: 0.9695 - val_loss: 0.3972 - val_accuracy: 0.8713


Decrese the tokens -> Worked. Overfitting decreased.

In [9]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model3'
HISTORY_NAME = 'history3'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))

#preprocess the data
train_dataset_vectorized2 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized2 = text_vectorization(test_dataset,vectorizer)

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
#x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)


#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 5)

Epoch 1/5
782/782 [==============================] - 113s 144ms/step - loss: 0.6419 - accuracy: 0.6891 - val_loss: 0.5628 - val_accuracy: 0.7730
Epoch 2/5
782/782 [==============================] - 6s 8ms/step - loss: 0.4719 - accuracy: 0.8341 - val_loss: 0.4248 - val_accuracy: 0.8457
Epoch 3/5
782/782 [==============================] - 5s 7ms/step - loss: 0.3641 - accuracy: 0.8722 - val_loss: 0.3602 - val_accuracy: 0.8653
Epoch 4/5
782/782 [==============================] - 7s 8ms/step - loss: 0.3085 - accuracy: 0.8880 - val_loss: 0.3275 - val_accuracy: 0.8748
Epoch 5/5
782/782 [==============================] - 6s 7ms/step - loss: 0.2737 - accuracy: 0.9000 - val_loss: 0.3086 - val_accuracy: 0.8810


Try Flattening. Not worked Increased overfitting a lot.

In [10]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model4'
HISTORY_NAME = 'history4'

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.Flatten()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#create the callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 5)



Epoch 1/5
782/782 [==============================] - 78s 99ms/step - loss: 0.4349 - accuracy: 0.7898 - val_loss: 0.3087 - val_accuracy: 0.8706
Epoch 2/5
782/782 [==============================] - 6s 7ms/step - loss: 0.1890 - accuracy: 0.9312 - val_loss: 0.3167 - val_accuracy: 0.8681
Epoch 3/5
782/782 [==============================] - 5s 6ms/step - loss: 0.0875 - accuracy: 0.9773 - val_loss: 0.3518 - val_accuracy: 0.8628
Epoch 4/5
782/782 [==============================] - 5s 7ms/step - loss: 0.0337 - accuracy: 0.9960 - val_loss: 0.3921 - val_accuracy: 0.8612
Epoch 5/5
782/782 [==============================] - 6s 8ms/step - loss: 0.0133 - accuracy: 0.9992 - val_loss: 0.4283 - val_accuracy: 0.8604


Train model 3 for longer.

In [11]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model5'
HISTORY_NAME = 'history5'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))

#preprocess the data
train_dataset_vectorized2 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized2 = text_vectorization(test_dataset,vectorizer)

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
#x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 100,
                                                 callbacks = [es,mc,tb])

Epoch 1/100
782/782 [==============================] - 80s 100ms/step - loss: 0.6396 - accuracy: 0.6891 - val_loss: 0.5574 - val_accuracy: 0.7766
Epoch 2/100
782/782 [==============================] - 5s 6ms/step - loss: 0.4664 - accuracy: 0.8356 - val_loss: 0.4203 - val_accuracy: 0.8472
Epoch 3/100
782/782 [==============================] - 6s 8ms/step - loss: 0.3601 - accuracy: 0.8730 - val_loss: 0.3573 - val_accuracy: 0.8666
Epoch 4/100
782/782 [==============================] - 4s 6ms/step - loss: 0.3055 - accuracy: 0.8882 - val_loss: 0.3255 - val_accuracy: 0.8758
Epoch 5/100
782/782 [==============================] - 6s 8ms/step - loss: 0.2711 - accuracy: 0.9010 - val_loss: 0.3071 - val_accuracy: 0.8812
Epoch 6/100
782/782 [==============================] - 5s 6ms/step - loss: 0.2460 - accuracy: 0.9104 - val_loss: 0.2959 - val_accuracy: 0.8837
Epoch 7/100
782/782 [==============================] - 6s 8ms/step - loss: 0.2260 - accuracy: 0.9179 - val_loss: 0.2894 - val_accuracy: 0.8

Add dropout and Dense layer. Change Activation to swish

In [12]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model6'
HISTORY_NAME = 'history6'
ACTIVATION = 'swish'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))

#preprocess the data
train_dataset_vectorized2 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized2 = text_vectorization(test_dataset,vectorizer)

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation = ACTIVATION)(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 100,
                                                 callbacks = [es,mc,tb])

Epoch 1/100
782/782 [==============================] - 78s 98ms/step - loss: 0.5004 - accuracy: 0.7428 - val_loss: 0.3224 - val_accuracy: 0.8728
Epoch 2/100
782/782 [==============================] - 5s 6ms/step - loss: 0.2618 - accuracy: 0.8980 - val_loss: 0.2858 - val_accuracy: 0.8878
Epoch 3/100
782/782 [==============================] - 5s 7ms/step - loss: 0.2070 - accuracy: 0.9220 - val_loss: 0.2886 - val_accuracy: 0.8873
Epoch 4/100
782/782 [==============================] - 5s 6ms/step - loss: 0.1736 - accuracy: 0.9356 - val_loss: 0.3090 - val_accuracy: 0.8835
Epoch 5/100
782/782 [==============================] - 5s 7ms/step - loss: 0.1495 - accuracy: 0.9444 - val_loss: 0.3419 - val_accuracy: 0.8774
Epoch 6/100
782/782 [==============================] - 5s 6ms/step - loss: 0.1292 - accuracy: 0.9544 - val_loss: 0.3807 - val_accuracy: 0.8734
Epoch 7/100
782/782 [==============================] - 5s 7ms/step - loss: 0.1122 - accuracy: 0.9614 - val_loss: 0.4261 - val_accuracy: 0.87

NOTE: LSTM does not work in GPU when it is used with relu;

In [13]:
#!tensorboard dev upload --logdir /content/TensorboardLogs

In [14]:
#clone the model
final_model = tf.keras.models.clone_model(models['model5'])

#load the best weights
final_model.load_weights("/content/ModelCheckpoints/model5.ckpt")

#compile the model
final_model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#evaluate the test dataset
final_model.evaluate(test_dataset_vectorized2)

782/782 [==============================] - 2s 2ms/step - loss: 0.2853 - accuracy: 0.8875


[0.28529679775238037, 0.8875200152397156]

In [15]:
#!tensorboard dev list

In [16]:
#!tensorboard dev delete --experiment_id rvOlSmxeQMiBvT4XhMmLdw

In [17]:
#final_model.save('final_model.h5')